### Importing necessary functions

In [ ]:
from random import randrange
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,TrainingArguments,pipeline
from peft import LoraConfig
from trl import SFTTrainer

### Loading training and test dataset

In [17]:
from datasets import load_dataset

ds = load_dataset("knkarthick/samsum")

In [18]:
# Convert the train split to a pandas DataFrame and display the first few rows
df_train = ds['train'].to_pandas()
df_train.head()

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\nJ...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\nKim: Bad mood tbh, I was ...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\nSam: i...,"Sam is confused, because he overheard Rick com..."


In [19]:
df_test = ds['test'].to_pandas()
df_test.head()

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\nRob: That's so gr8!\nEric: I k...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\n...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\nJane: yes, just...",Jane is in Warsaw. Ollie and Jane has a party....


In [20]:
model_name = "google/flan-t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#Makes training faster but a little less accurate 
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

#setting padding instructions for tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [21]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response:

    ### Task:
    Summarize the Input

    ### Input:
    {sample['dialogue']}

    ### Response:
    {sample['summary']}
    """ 

In [22]:
# Create the trainer
trainingArgs = TrainingArguments(
    output_dir='output',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_strategy="epoch",
    learning_rate=2e-4
)

peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=df_train,
    eval_dataset = df_test,
    peft_config=peft_config,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

trainer.train()

c:\Users\MarijaJolovic\miniconda3\envs\myenv1\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': packing. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\MarijaJolovic\miniconda3\envs\myenv1\Lib\site-packages\trl\trainer\sft_trainer.py:212: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\MarijaJolovic\miniconda3\envs\myenv1\Lib\site-packages\trl\trainer\sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 512
  warnings.warn(
Generating train split: 0 examples [00:00, ? examples/s]


ValueError: Error occurred while packing the dataset. Make sure that your dataset has enough samples to at least yield one packed sequence.

In [ ]:
trainer.train()

In [24]:
# define the summarizer pipeline
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# select a random test sample
sample = df_test.iloc[randrange(len(df_test))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-small summary:\n{res[0]['summary_text']}")
print(f"true summary:\n{sample['summary']}")

Your max_length is set to 200, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


dialogue: 
Alex: Hon, could you buy some milk?
Martin: You kiddin me, I just bought two yesterday
Alex: Well, I have nothing for my defense, I'm just craving milk lately:D
Martin: You're lucky that I love you, two milks coming up in couple of hours, I gotta finish the project
Alex: You are my hero!
---------------
flan-t5-small summary:
Martin bought two milks yesterday. Alex has nothing for his defense. Alex is a hero. Alex wants to finish the project. Alex and Alex will buy milk in two hours.
true summary:
Martin will get Alex two bottles of milk after he finishes the project.
